In [1]:
import pandas as pd
import re

In [24]:
# EXPAND marketnoticedata
old_df = pd.read_csv("noticedata1998.csv")
new_df = pd.read_csv("recentnoticedata.csv",header=old_df.columns)
df = old_df.merge(new_df,how='outer')
df.set_index('EFFECTIVEDATE',inplace=True)
df.index = pd.to_datetime(df.index)
df.sort_index(inplace=True)
df.to_csv("noticedata1998.csv")

In [41]:
# dtypes = {'col1': 'str', 'col2': 'str', 'col3': 'str', 'col4': 'float'}
df = pd.read_csv("noticedata1998.csv",parse_dates=['EFFECTIVEDATE'])
df['EXTERNALREFERENCE'] = df['EXTERNALREFERENCE'].astype(str)
df.set_index('EFFECTIVEDATE',inplace=True)
# df.info()
# df = df["2024-01-01":]
# df.tail()

### Documentation

A general summary of market notices will not be possible with this tool, as it was decided that general summaries of directions are not helpful. In a sense, the TypeID will give the user a general summary of what they are looking at. The key to using this tool will be to use the TOGGLE column, which will offer options to toggle which section of the market notices you are looking at. For example, toggling LOR will provide the user with the portion of market notices that pertain to lack of reserve directions. Then, toggling Power System Security will give all notices relating to power system security and system strength. In most cases, it will not make sense to toggle more than one, although there is no proof of this yet.  

After doing so, it will be possible to navigate to the corresponding DIRTYPE for further analysis.

The table which reflects the choices made by the user will be downloadable. Therefore, this tool creates an easy way for the user to make sense of marketnoticedata by downloading tables and figures that are only relevant to what is being investigated by the user.

The toggles are:

- PSS: Power System Security
- MSL: Minimum System Load
- LOR: Lack of Reserve
- RERT: Reliability and Emergency Reserve Trader

The hope is to add values for the amount of RERT dispatched as well, though it may be challenging.

The MSL were only recently being tagged in the EXTERNALREFERENCE column as being actuals so the cancellations were checked as well. If the cancellation was after the forecast, it was decided that the MSL forecast actually happened as so it should be tagged as actual. This was inspected manually in excel.

In [ ]:
#PSS 
pssdirtypes = ('Cancellation','Update','Possible Intervention','Direction')
pssdirs = ('Cancellation','Update','Possible Intervention','Direction')
df['PSSDIRTYPES'] = df['EXTERNALREFERENCE'].apply(lambda x: next((pssdirs[index] for index, psstype in enumerate(pssdirtypes) \
                   if pd.Series(x).str.contains(psstype,case=False,regex=True).any()), 'N/A'))

psstypes = # tbd
pssnames = # tbd
df['PSSTYPES'] =  df['EXTERNALREFERENCE'].apply(lambda x: next((pssnames[index] for index, psstype in enumerate(psstypes) \
                   if pd.Series(x).str.contains(psstype,case=False,regex=True).any()), 'N/A'))

In [44]:
#LOR forecasts
lordirtypes = ('amend','revis','upadte|update','correction','cancellation','actual','forecast')
lordirs = ('Amendment','Revision','Update','Correction','Cancellation','Actual','Forecast')
df['LORDIRTYPES'] = df['EXTERNALREFERENCE'].apply(lambda x: next((lordirs[index] for index, lortype in enumerate(lordirtypes) \
                   if pd.Series(x).str.contains(lortype,case=False,regex=True).any()), 'N/A'))

lortypes = ('lrc|lack of reserve condition','lor1|lor 1','lor2|lor 2','lor3|lor 3')
lornames = ('LRC','LOR1','LOR2','LOR3')
df['LORTYPES'] =  df['EXTERNALREFERENCE'].apply(lambda x: next((lornames[index] for index, lortype in enumerate(lortypes) \
                   if pd.Series(x).str.contains(lortype,case=False,regex=True).any()), 'N/A'))

MSL

In [ ]:
#MSL 
msldirtypes = ('cancellation','forecast','direction')
msldirs = ('Cancellation','Forecast','Direction')
df['LORDIRTYPES'] = df['EXTERNALREFERENCE'].apply(lambda x: next((lordirs[index] for index, lortype in enumerate(lordirtypes) \
                   if pd.Series(x).str.contains(lortype,case=False,regex=True).any()), 'N/A'))

# msltypes = ('lrc|lack of reserve condition','lor1|lor 1','lor2|lor 2','lor3|lor 3')
# lornames = ('LRC','LOR1','LOR2','LOR3')
# df['LORTYPES'] =  df['EXTERNALREFERENCE'].apply(lambda x: next((lornames[index] for index, lortype in enumerate(lortypes) \
#                    if pd.Series(x).str.contains(lortype,case=False,regex=True).any()), 'N/A'))

Can plug holes in by looking at RERT activation figures. E.g. 2016 listed as actual LOR2 and not as RERT. 

In [ ]:
#RERT 
rertdirtypes = ('cancellation','forecast','direction')
msldirs = ('Cancellation','Forecast','Direction')
df['LORDIRTYPES'] = df['EXTERNALREFERENCE'].apply(lambda x: next((lordirs[index] for index, lortype in enumerate(lordirtypes) \
                   if pd.Series(x).str.contains(lortype,case=False,regex=True).any()), 'N/A'))

# msltypes = ('lrc|lack of reserve condition','lor1|lor 1','lor2|lor 2','lor3|lor 3')
# lornames = ('LRC','LOR1','LOR2','LOR3')
# df['LORTYPES'] =  df['EXTERNALREFERENCE'].apply(lambda x: next((lornames[index] for index, lortype in enumerate(lortypes) \
#                    if pd.Series(x).str.contains(lortype,case=False,regex=True).any()), 'N/A'))

In [ ]:
df = pd.read_csv("rert_dispatch_amount.csv")

df.head()

: 

In [43]:
places = ('inter regional|inter-regional','SA|south australia','VIC|victoria','NSW|new south wales','QLD|queensland','TAS|tasmania')
names = ('Inter-regional','SA','VIC','NSW','QLD','TAS')

# not completely accurate since does not account for multiple state mentions but close enough
df['REGION']=df['REASON'].apply(lambda x: next((names[index] for index, place in enumerate(places) \
                   if pd.Series(x).str.contains(place,case=False,regex=True).any()), 'N/A'))

# rough heuristic in order of how they appear, can work on this tagging
dirtypes = ('Cancellation','Update','Possible Intervention','Direction')
df['DIRTYPE']=df['EXTERNALREFERENCE'].apply(lambda x: next((name for name in dirtypes \
                   if pd.Series(x).str.contains(name,case=False,regex=True).any()), 'N/A'))

# another method
# df['DIRTYPE']=df['REASON'].apply(lambda x: next((name for name in dirtypes \
#                    if pd.Series(x).str.contains(name,case=False,regex=True).any()), 'N/A'))

print(df.tail())

if False:
    #dataset['SA']=dataset['EXTERNALREFERENCE'].str.extract(r'(SA|south australia)',flags=re.IGNORECASE,expand=False)
    #dataset['VIC']=dataset['EXTERNALREFERENCE'].str.extract(r'(VIC|victoria)',flags=re.IGNORECASE,expand=False)
    #dataset['NSW']=dataset['EXTERNALREFERENCE'].str.extract(r'(NSW|new south wales)',flags=re.IGNORECASE,expand=False)
    #dataset['QLD']=dataset['EXTERNALREFERENCE'].str.extract(r'(QLD|queensland)',flags=re.IGNORECASE,expand=False)
    #dataset['TAS']=dataset['EXTERNALREFERENCE'].str.extract(r'(TAS|tasmania)',flags=re.IGNORECASE,expand=False)

    #extractall(r'(inter)(SA|south australia)(VIC|victoria)(NSW|new south wales)(QLD|queensland)(TAS|tasmania)',flags=re.IGNORECASE).groupby(level=0).agg(''.join)
    #dataset['LOR']=dataset['EXTERNALREFERENCE'].str.extract(r'(lor1|lor2|lor3)',flags=re.IGNORECASE,expand=False)
    #dataset['DIRECTION']=dataset['EXTERNALREFERENCE'].str.extract(r'(direction)',flags=re.IGNORECASE,expand=False)
    #dataset['CANCEL']=dataset['EXTERNALREFERENCE'].str.extract(r'(cancel)',flags=re.IGNORECASE,expand=False)
    #dataset['POSSIBLE INTERVENTION']=dataset['EXTERNALREFERENCE'].str.extract(r'(possible intervention)',flags=re.IGNORECASE,expand=False)

    #dataset['INTERSTATE']=dataset['EXTERNALREFERENCE'].str.extract(r'(inter)',flags=re.IGNORECASE,expand=False)
    #dataset['KEYWORDS'].apply(lambda x: next((provider for provider in pattern \
    #                                                     if pd.Series(x).str.contains(provider, case=False, regex=True).any()), \
    #                                                     'Other))
    # make keyword thing for everything in order of priority, RERT, LOR3, cancellation,direction,interstate,states,possible intervention,LORs
    # make groups cancellation, direction, possible intervention
    # interstate, states
    # LORs
    # states
    # system strength / power system security
    # RERT
    print("x")


                     NOTICEID               TYPEID NOTICETYPE  \
EFFECTIVEDATE                                                   
2024-11-28 14:24:36    121170       RESERVE NOTICE     MARKET   
2024-11-28 14:26:09    121171       RESERVE NOTICE     MARKET   
2024-11-28 14:29:51    121165  MARKET INTERVENTION     MARKET   
2024-11-28 14:59:59    121166  MARKET INTERVENTION     MARKET   
2024-11-28 15:01:25    121167  MARKET INTERVENTION     MARKET   

                             LASTCHANGED  \
EFFECTIVEDATE                              
2024-11-28 14:24:36  2024-11-28 14:24:36   
2024-11-28 14:26:09  2024-11-28 14:26:09   
2024-11-28 14:29:51  2024-11-28 14:29:51   
2024-11-28 14:59:59  2024-11-28 14:59:59   
2024-11-28 15:01:25  2024-11-28 15:01:25   

                                                                REASON  \
EFFECTIVEDATE                                                            
2024-11-28 14:24:36  AEMO ELECTRICITY MARKET NOTICE  The Forecast L...   
2024-11-28 14

In [46]:
df.dropna()

,NOTICEID,TYPEID,NOTICETYPE,LASTCHANGED,REASON,EXTERNALREFERENCE,LORTYPES,REGION,DIRTYPE,LORDIRTYPES
EFFECTIVEDATE,,,,,,,,,,
1998-10-29 10:13:36,1,VOLL,MARKET,1998-10-29 10:13:36,Testing of Market Systems A manual override o...,nan,N/A,N/A,N/A,N/A
1998-10-30 10:13:05,2,MARKET SYSTEMS,MARKET,1998-10-30 10:13:05,Following changes to the firewall last night a...,Firewall problems,N/A,N/A,N/A,N/A
1998-11-03 10:58:18,501,TEST,MARKET,1998-11-03 10:58:18,Test of market notices from 500 up,Test,N/A,N/A,N/A,N/A
1998-11-03 14:54:03,503,TEST,MARKET,1998-11-03 14:54:03,This is a test message.,nan,N/A,N/A,N/A,N/A
1998-11-08 11:02:30,505,MARKET SYSTEMS,MARKET,1998-11-08 11:02:30,the nemnet connection is failing over to the b...,nemnet,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...
2024-11-28 14:24:36,121170,RESERVE NOTICE,MARKET,2024-11-28 14:24:36,AEMO ELECTRICITY MARKET NOTICE The Forecast L...,PDPASA - Update of the Forecast Lack Of Reserv...,LOR1,SA,Update,Update
2024-11-28 14:26:09,121171,RESERVE NOTICE,MARKET,2024-11-28 14:26:09,AEMO ELECTRICITY MARKET NOTICE The Forecast L...,PDPASA - Update of the Forecast Lack Of Reserv...,LOR2,SA,Update,Update
2024-11-28 14:29:51,121165,MARKET INTERVENTION,MARKET,2024-11-28 14:29:51,AEMO ELECTRICITY MARKET NOTICE Cancellation: ...,Cancellation: Direction issued to: AGL SA Gen...,N/A,SA,Cancellation,Cancellation


In [47]:
df

,NOTICEID,TYPEID,NOTICETYPE,LASTCHANGED,REASON,EXTERNALREFERENCE,LORTYPES,REGION,DIRTYPE,LORDIRTYPES
EFFECTIVEDATE,,,,,,,,,,
1998-10-29 10:13:36,1,VOLL,MARKET,1998-10-29 10:13:36,Testing of Market Systems A manual override o...,nan,N/A,N/A,N/A,N/A
1998-10-30 10:13:05,2,MARKET SYSTEMS,MARKET,1998-10-30 10:13:05,Following changes to the firewall last night a...,Firewall problems,N/A,N/A,N/A,N/A
1998-11-03 10:58:18,501,TEST,MARKET,1998-11-03 10:58:18,Test of market notices from 500 up,Test,N/A,N/A,N/A,N/A
1998-11-03 14:54:03,503,TEST,MARKET,1998-11-03 14:54:03,This is a test message.,nan,N/A,N/A,N/A,N/A
1998-11-08 11:02:30,505,MARKET SYSTEMS,MARKET,1998-11-08 11:02:30,the nemnet connection is failing over to the b...,nemnet,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...
2024-11-28 14:24:36,121170,RESERVE NOTICE,MARKET,2024-11-28 14:24:36,AEMO ELECTRICITY MARKET NOTICE The Forecast L...,PDPASA - Update of the Forecast Lack Of Reserv...,LOR1,SA,Update,Update
2024-11-28 14:26:09,121171,RESERVE NOTICE,MARKET,2024-11-28 14:26:09,AEMO ELECTRICITY MARKET NOTICE The Forecast L...,PDPASA - Update of the Forecast Lack Of Reserv...,LOR2,SA,Update,Update
2024-11-28 14:29:51,121165,MARKET INTERVENTION,MARKET,2024-11-28 14:29:51,AEMO ELECTRICITY MARKET NOTICE Cancellation: ...,Cancellation: Direction issued to: AGL SA Gen...,N/A,SA,Cancellation,Cancellation


In [48]:
df.to_parquet("noticeDataWithTags.parquet")

In [50]:
# work on a better tagging system. -> probably just picking up more common tags to reduce N/A count

# worth adding Actual to Dirtype? probably

# do regions by REASON to get more accurate results for RERT especially

# don't overdo it! I think that's enough - Arthur

# add new "Other Types" field (see below)

# activated, intends, actual, forecast, etc

# LOR, RERT - can get amount dispatched fromDISPATCHLOAD or DISPATCH_UNIT_SCADAt, WDR dashboard

# some obvious ones: manual overrides (manually priced dispatch intervals), MTPASA ??, inter regional transfer limit, power system outage, power system security, system strength, non-conformance